# 카카오 도로명 주소 API

In [ ]:
import folium
import json
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]
filename

Saving 카톡_api_key.txt to 카톡_api_key.txt


'카톡_api_key.txt'

In [ ]:
from google.colab import files
uploaded = files.upload()
filename1 = list(uploaded.keys())[0]
filename1

In [ ]:
with open(filename) as f:
    api_key = f.read()

In [ ]:
with open(filename1) as f:
    r_api_key = f.read()

In [ ]:
import requests
from urllib.parse import quote

In [ ]:
addr = '서울특별시 종로구 종로1길 36(수송동)'
quote(addr)

In [ ]:
search_url = "https://dapi.kakao.com/v2/local/search/address.json"
url = f'{search_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%A2%85%EB%A1%9C%EA%B5%AC%20%EC%A2%85%EB%A1%9C1%EA%B8%B8%2036%28%EC%88%98%EC%86%A1%EB%8F%99%29'

In [ ]:
result = requests.get(url, headers={"Authorization": f"KakaoAK {api_key}"}).json()

In [ ]:
result

{'documents': [{'address': {'address_name': '서울 종로구 수송동 146-12',
    'b_code': '1111012400',
    'h_code': '1111061500',
    'main_address_no': '146',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '종로구',
    'region_3depth_h_name': '종로1.2.3.4가동',
    'region_3depth_name': '수송동',
    'sub_address_no': '12',
    'x': '126.978988255925',
    'y': '37.5735051436739'},
   'address_name': '서울 종로구 종로1길 36',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 종로구 종로1길 36',
    'building_name': '종로구청',
    'main_building_no': '36',
    'region_1depth_name': '서울',
    'region_2depth_name': '종로구',
    'region_3depth_name': '수송동',
    'road_name': '종로1길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.978988255925',
    'y': '37.5735051436739',
    'zone_no': '03152'},
   'x': '126.978988255925',
   'y': '37.5735051436739'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [ ]:
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])

In [ ]:
df = pd.read_csv('./서울시_주요기관.csv')

In [ ]:
df

,장소,도로명주소,위도,경도
0,종로구청,종로구청,37.573505,126.978988
1,노원구청,노원구청,37.654519,127.056297
2,송파구청,송파구청,37.514477,127.10586
3,마포구청,마포구청,37.563426,126.903357
4,양천구청,양천구청,37.517075,126.866543
5,도봉구청,도봉구청,37.668716,127.047131


In [ ]:
df['위도'] = ""
df['경도'] = ""
for i in df.index:
    url = f'{search_url}?query={quote(df.도로명주소[i])}'
    result = requests.get(url, headers={"Authorization": f"KakaoAK {api_key}"}).json()
    lng = float(result['documents'][0]['x'])
    lat = float(result['documents'][0]['y'])
    df.loc[i] = [df.loc[i, '장소'], df.loc[i, '도로명주소'], lat, lng]
df

,장소,도로명주소,위도,경도
0,종로구청,서울특별시 종로구 종로1길 36(수송동),37.573505,126.978988
1,노원구청,서울특별시 노원구 노해로 437(상계동),37.654519,127.056297
2,송파구청,서울특별시 송파구 올림픽로 326(신천동),37.514477,127.10586
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동),37.563426,126.903357
4,양천구청,서울특별시 양천구 목동동로 105(신정동),37.517075,126.866543
5,도봉구청,서울특별시 도봉구 마들로 656(방학동),37.668716,127.047131


In [ ]:
df.to_csv('공공기관2.csv', index=False)

In [ ]:
# 1. 공공기관 지도표시
# 2. 본인고향의 관공서 지도 표시
# 3. 부모님 고향 관공서

In [ ]:
# 지도 만들기
map = folium.Map(location=[37.5502, 126.982], zoom_start=10)

# 1. 공공기관 지도표시
for i in df.index:
    folium.CircleMarker(
        radius=20,
        location = [df.위도[i], df.경도[i]], 
        popup = folium.Popup(df.도로명주소[i], max_width=200),
        tooltip = df.장소, 
        color = 'red',
        fill=True,
        fill_color = 'crimson',
    ).add_to(map)

# 2. 본인고향의 관공서 지도 표시
bldg = '인천광역시 연수구청'
base_url = "https://business.juso.go.kr/addrlink/addrLinkApi.do"
params1 = f'currentaPage=1&countPerPage=10&confmKey={r_api_key}'
params2 = f'keyword={quote(bldg)}&resultType=json'
url = f'{base_url}?{params1}&{params2}'
result = requests.get(url)
res = json.loads(result.text)
road_add = res['results']['juso'][0]['roadAddr']
url = f'{search_url}?query={quote(road_add)}'
result = requests.get(url, headers={"Authorization": f"KakaoAK {api_key}"}).json()
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])
folium.CircleMarker(
        radius=20,
        location = [lat, lng], 
        popup = folium.Popup(road_add, max_width=200),
        tooltip = bldg, 
        color = 'red',
        fill=True,
        fill_color = 'crimson',
    ).add_to(map)

# 3. 부모님 고향 관공서
bldg = '파주시청'
base_url = "https://business.juso.go.kr/addrlink/addrLinkApi.do"
params1 = f'currentaPage=1&countPerPage=10&confmKey={r_api_key}'
params2 = f'keyword={quote(bldg)}&resultType=json'
url = f'{base_url}?{params1}&{params2}'
result = requests.get(url)
res = json.loads(result.text)
road_add = res['results']['juso'][0]['roadAddr']
url = f'{search_url}?query={quote(road_add)}'
result = requests.get(url, headers={"Authorization": f"KakaoAK {api_key}"}).json()
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])
folium.CircleMarker(
        radius=20,
        location = [lat, lng], 
        popup = folium.Popup(road_add, max_width=200),
        tooltip = bldg, 
        color = 'red',
        fill=True,
        fill_color = 'crimson',
    ).add_to(map)

# 4. 원하는 관공서
bldg = input('지도에 표시하고 싶은 관공서를 입력하세요 : ')
base_url = "https://business.juso.go.kr/addrlink/addrLinkApi.do"
params1 = f'currentaPage=1&countPerPage=10&confmKey={r_api_key}'
params2 = f'keyword={quote(bldg)}&resultType=json'
url = f'{base_url}?{params1}&{params2}'
result = requests.get(url)
res = json.loads(result.text)
road_add = res['results']['juso'][0]['roadAddr']
url = f'{search_url}?query={quote(road_add)}'
result = requests.get(url, headers={"Authorization": f"KakaoAK {api_key}"}).json()
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])
folium.CircleMarker(
        radius=20,
        location = [lat, lng], 
        popup = folium.Popup(road_add, max_width=200),
        tooltip = bldg, 
        color = 'red',
        fill=True,
        fill_color = 'crimson',
    ).add_to(map)

# 제목 지정 및 지도 출력
title = '<h3 align="center" style="font-size:20[x">대한민국 관공서 지도</h3>'
map.get_root().html.add_child(folium.Element(title))
map

지도에 표시하고 싶은 관공서를 입력하세요 : 고양시청
